<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Gradient Descent in Sklearn

_Authors: Kiefer Katovich (SF)_

---

Until now we've been using specific sklearn model classes to perform regression and classification such as `LinearRegression` and `LogisticRegression`. Unfortunately, while these methods work well on smaller datasets with relatively small numbers of columns, once you start getting into "Medium Data" these slow down to a crawl, and take up so much memory that fitting them becomes mind-numbingly slow (especially on a laptop).

Luckily, sklearn comes with  stochastic gradient descent solvers for regression and classification:
- `SGDRegressor`
- `SGDClassifier`

Due to its ability to minimize the loss function iteratively on smaller portions of the data, it avoids the intense slowdown other models suffer on large datasets.

> **Note:** The gradient descent solvers are very flexible and can fit a variety of different model types not covered here. I highly recommend reading their documentation in detail.

---

### SF assessor data

This lab uses data from the SF assessor's office on housing prices in San Francisco - it's already cleaned up.

You can see that the dataset has 250k rows. When expanding this with dummy-coded categorical columns it can become quite large. Be careful that you don't exceed the memory on your computer.


In [29]:
import numpy as np
import scipy 
import seaborn as sns
import pandas as pd
import scipy.stats as stats

import patsy

import matplotlib
import matplotlib.pyplot as plt

%config InlineBackend.figure_format = 'retina'
%matplotlib inline

plt.style.use('fivethirtyeight')

### 1. Load the data.

Examine the columns.

In [2]:
prop = pd.read_csv('./datasets/assessor_sample.csv')

In [3]:
prop.shape

(754147, 17)

In [4]:
prop.dtypes

baths               int64
beds                int64
lot_depth         float64
basement_area     float64
front_ft          float64
owner_pct         float64
rooms               int64
property_class     object
neighborhood       object
tax_rate          float64
volume              int64
sqft                int64
stories             int64
year_recorded       int64
year_built          int64
zone               object
value             float64
dtype: object

In [6]:
prop.isnull().sum()

baths             0
beds              0
lot_depth         0
basement_area     0
front_ft          0
owner_pct         0
rooms             0
property_class    0
neighborhood      0
tax_rate          0
volume            0
sqft              0
stories           0
year_recorded     0
year_built        0
zone              0
value             0
dtype: int64

In [30]:
prop.property_class.unique()

array(['Z', 'D', 'ZBM', 'DBM', 'LZ', 'TH'], dtype=object)

In [31]:
prop.neighborhood.unique()

array(['08E', '08G', '08D', '08H', '08C', '08B', '08A', '08I', '07A',
       '07D', '07B', '08F', '06C', '06D', '06E', '06B', '06A', '07C',
       '01C', '01G', '06F', '05B', '05F', '05E', '05H', '01F', '01D',
       '01E', '01A', '01B', '02C', '02E', '02F', '02A', '02B', '02D',
       '02G', '04C', '04T', '03C', '05J', '05G', '04D', '05K', '05D',
       '04F', '05C', '04J', '04H', '047', '04K', '04G', '04P', '04S',
       '04M', '04N', '04R', '10H', '04A', '05M', '09F', '09C', '09H',
       '09B', '09D', '09E', '10J', '10A', '10K', '10G', '10F', '09A',
       '09G', '10C', '10E', '10B', '05A', '04B', '04E', '03H', '10D',
       '03B', '03J', '03G', '03F', '03A', '03E', '03D'], dtype=object)

### 2. Sample down the data

Despite this already being a sample of the full assessor dataset, you should sample the data down further the sake of speed and your computers memory.

Use the `.sample()` function for pandas dataframes to subset this down to < 25000 rows. 

Sampling down large datasets is a common procedure. Finding the optimal parameters with larger subsets of the data may change the hyperparameters and the results, and will get you closer to the best coefficients, but the returns are marginal at a point.

In [34]:
prop_samp = prop.sample(n=25000)

### 3. Regression with stochastic gradient descent

Below I set up X, y data predicting value (housing price) from the remaining variables. There are ~75,000 rows, with 170 columns.


The `SGDRegressor` is very general and flexible, and can be customized with a variety of keyword arguments.

**Arguments**
- `loss`: `['squared_loss','huber', ...]`
    - The `'squared_loss'` loss corresponds to solving a regression with the least squares loss. This is what I expect you'll use, but there are other options. Huber loss is a "robust" regression loss.
- `penalty`: `['none','l1','l2','elasticnet']`
    - This defines the penalty on the regression that you would like to solve. The l1 and l2 are the Lasso and Ridge, while the elasticnet is the combination of them both.
- `alpha`
    - The regularization strength to be used with a chosen penalty. Same as in Lasso and Ridge.
- `l1_ratio`
    - The mix of the Lasso and Ridge penalties when elasticnet is chosen as the penalty.
- `n_iter`
    - The number of training "epochs" over the data. This is the number of passes that the gradient descent algorithm will make over the data to iteratively fit the weights (defaults to 5).

`SGDRegressor` is most often used in tandem with grid searching to find the optimal parameters for certain models. 

**It is up to you how you want to define the model. You should:**

1. Choose a target to estimate (this should be continuous).
- Select predictors to use.
- Standardize your predictor matrix.
- Build a stochastic gradient descent solver to fit your model. You will likely want to do some kind of gridsearch to find the optimal parameters for your model.
- Describe the model selected through gridsearch and compare the performance to baseline.
- Examine and interpret the coefficients.

In [35]:
f = 'value ~ '+' + '.join([c for c in prop_samp.columns if not c == 'value'])+' -1'
print f
y, X = patsy.dmatrices(f, data=prop_samp, return_type='dataframe')
y = y.values.ravel()

print y.shape, X.shape

value ~ baths + beds + lot_depth + basement_area + front_ft + owner_pct + rooms + property_class + neighborhood + tax_rate + volume + sqft + stories + year_recorded + year_built + zone -1
(25000,) (25000, 163)


In [36]:
# I am predicting the value of the house from the other variables.

In [37]:
from sklearn.linear_model import SGDRegressor, SGDClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.grid_search import GridSearchCV

In [38]:
# standardize the predictors
ss = StandardScaler()
Xs = ss.fit_transform(X)
print Xs.shape

(25000, 163)


In [42]:
# set up my gridsearch parameters:
sgd_params = {
    'loss':['squared_loss','huber'],
    'penalty':['l1','l2'],
    'alpha':np.logspace(-5,1,25)
}

sgd_reg = SGDRegressor()
sgd_reg_gs = GridSearchCV(sgd_reg, sgd_params, cv=5, verbose=1)

In [43]:
# SGD is pretty fast compared to other sklearn solvers - but can still
# take a good long while depending on the gridsearch and the size of
# the dataset.
sgd_reg_gs.fit(Xs, y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:  1.1min finished


GridSearchCV(cv=5, error_score='raise',
       estimator=SGDRegressor(alpha=0.0001, average=False, epsilon=0.1, eta0=0.01,
       fit_intercept=True, l1_ratio=0.15, learning_rate='invscaling',
       loss='squared_loss', n_iter=5, penalty='l2', power_t=0.25,
       random_state=None, shuffle=True, verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'loss': ['squared_loss', 'huber'], 'alpha': array([  1.00000e-05,   1.77828e-05,   3.16228e-05,   5.62341e-05,
         1.00000e-04,   1.77828e-04,   3.16228e-04,   5.62341e-04,
         1.00000e-03,   1.77828e-03,   3.16228e-03,   5.62341e-03,
         1.00000e-...2341e-01,
         1.00000e+00,   1.77828e+00,   3.16228e+00,   5.62341e+00,
         1.00000e+01])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [44]:
print sgd_reg_gs.best_params_
print sgd_reg_gs.best_score_
sgd_reg = sgd_reg_gs.best_estimator_

{'penalty': 'l2', 'alpha': 0.56234132519034907, 'loss': 'squared_loss'}
0.227204132061


In [45]:
# We can see that the gradient descent got a .22 R2 using the ridge and squared loss.

In [48]:
value_coefs = pd.DataFrame({'coef':sgd_reg.coef_,
                            'mag':np.abs(sgd_reg.coef_),
                            'pred':X.columns})
value_coefs.sort_values('mag', ascending=False, inplace=True)
value_coefs.iloc[0:10, :]

,coef,mag,pred
151,45564.228979,45564.228979,beds
159,32631.977463,32631.977463,sqft
70,28652.346605,28652.346605,neighborhood[T.08E]
155,-28189.115056,28189.115056,owner_pct
161,24717.889344,24717.889344,year_recorded
52,23467.877598,23467.877598,neighborhood[T.05H]
62,23101.318589,23101.318589,neighborhood[T.07A]
150,23091.664374,23091.664374,baths
63,23034.538664,23034.538664,neighborhood[T.07B]
47,22842.291333,22842.291333,neighborhood[T.05C]


### 4. Classification with stochastic gradient descent

The `SGDClassifier` is very similar to the `SGDRegressor`. The main difference is that the loss functions are changed to regression loss functions.

**Arguments**
- `loss`: `['log', ...]`
    - The `'log'` loss corresponds to solving a logistic regression classifier. This is what I expect you'll use, but there are many other options.
- `penalty`: `['none','l1','l2','elasticnet']`
    - This defines the penalty on the regression that you would like to solve. The l1 and l2 are the Lasso and Ridge, while the elasticnet is the combination of them both.
- `alpha`
    - The regularization strength to be used with a chosen penalty. Same as in Lasso and Ridge.
- `l1_ratio`
    - The mix of the Lasso and Ridge penalties when elasticnet is chosen as the penalty.
- `n_iter`
    - The number of training "epochs" over the data. This is the number of passes that the gradient descent algorithm will make over the data to iteratively fit the weights (defaults to 5).

Like `SGDRegressor`, `SGDClassifier` is most often used in tandem with grid searching to find the optimal parameters for certain models. 

**It is up to you how you want to define the model. You should:**

1. Choose a target to classify (you may need to engineer one from existing variables).
- Calculate the baseline accuracy.
- Select predictors to use.
- Standardize your predictor matrix.
- Build a stochastic gradient descent solver to fit your model. You will likely want to do some kind of gridsearch to find the optimal parameters for your model.
- Describe the model selected through gridsearch and compare the performance to baseline.
- Examine and interpret the coefficients.

In [49]:
prop_samp.columns

Index([u'baths', u'beds', u'lot_depth', u'basement_area', u'front_ft',
       u'owner_pct', u'rooms', u'property_class', u'neighborhood', u'tax_rate',
       u'volume', u'sqft', u'stories', u'year_recorded', u'year_built',
       u'zone', u'value'],
      dtype='object')

In [54]:
prop_samp.year_built.value_counts()

1941    965
1940    927
1925    910
1924    801
1926    775
1927    731
1939    633
1923    623
1948    567
1947    530
1908    529
1928    500
1922    445
1938    444
1950    443
1906    419
1946    416
1907    402
1931    394
1951    389
1930    345
1910    344
1936    338
1949    337
1929    335
1944    333
1937    332
1912    305
1942    301
1945    272
       ... 
1996    112
1997    105
1933     96
1993     95
1998     87
1995     81
1985     81
1918     77
1994     74
1977     74
1976     70
1970     66
1902     65
1973     62
1966     61
1974     55
1968     49
2000     46
1967     46
1903     42
1934     34
1971     34
1969     30
1999     13
1901     11
2003      5
2009      3
2001      2
2004      2
2006      1
Name: year_built, dtype: int64

In [57]:
# lets see if we can predict if a house was built past 1980
prop_samp['built_past1980'] = prop_samp.year_built.map(lambda x: 1 if x >= 1980 else 0)

In [58]:
# make the target and calculate the baseline:
y = prop_samp.built_past1980.values
print 1. - np.mean(y)

0.89824


In [60]:
f = '''
~ baths + beds + lot_depth + basement_area + front_ft + owner_pct +
rooms + property_class + neighborhood + tax_rate + volume + sqft + stories +
zone + value -1
'''

In [61]:
X = patsy.dmatrix(f, data=prop_samp, return_type='dataframe')

Xs = ss.fit_transform(X)
print y.shape, Xs.shape

(25000,) (25000, 162)


In [62]:
sgd_cls_params = {
    'loss':['log'],
    'penalty':['l1','l2'],
    'alpha':np.logspace(-5,2,50)
}

sgd_cls = SGDClassifier()
sgd_cls_gs = GridSearchCV(sgd_cls, sgd_cls_params, cv=5, verbose=1)

In [63]:
sgd_cls_gs.fit(Xs, y)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   56.8s finished


GridSearchCV(cv=5, error_score='raise',
       estimator=SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='hinge', n_iter=5, n_jobs=1,
       penalty='l2', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=False),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'penalty': ['l1', 'l2'], 'loss': ['log'], 'alpha': array([  1.00000e-05,   1.38950e-05,   1.93070e-05,   2.68270e-05,
         3.72759e-05,   5.17947e-05,   7.19686e-05,   1.00000e-04,
         1.38950e-04,   1.93070e-04,   2.68270e-04,   3.72759e-04,
         5.17947e-04,   7.19686e-04,...    1.93070e+01,   2.68270e+01,   3.72759e+01,   5.17947e+01,
         7.19686e+01,   1.00000e+02])},
       pre_dispatch='2*n_jobs', refit=True, scoring=None, verbose=1)

In [64]:
print sgd_cls_gs.best_params_
print sgd_cls_gs.best_score_
sgd_cls = sgd_cls_gs.best_estimator_

{'penalty': 'l1', 'alpha': 7.1968567300115141e-05, 'loss': 'log'}
0.96464
